In [12]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle

import time
import numpy as np
import os

In [13]:
NUM_CLASSES = 2 
USE_IMAGE_GENERATOR = False
PIC_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 12

# Load data and data preprocessing 
## Step 1: Load data +++Shijei++++
load_data為自己定義function，目標是將data load 進來，output = (x_train, y_train)<br>
ex : x_train.shape = (data_size, 224, 224, 3)  y_train.shape =(data_size,)<br>

In [14]:
def load_data(source='dog'):
    if source is 'dog':
        image_path = "C:\\Users\jack_tsai\PycharmProjects\TensorTest" + "\\train"
        print("Loading from ", image_path)
        img_data_list = []
        y_label_list = []
        img_list = os.listdir(image_path)
        for img_name in img_list[12250:12750]:
            img_data_path = image_path + "\\" + img_name
            img = image.load_img(img_data_path, target_size=(PIC_SIZE, PIC_SIZE))
            img_array = image.img_to_array(img)
            img_data_list.append(img_array)
            if 'cat' in img_name:
                y_label_list.append(0)
            elif 'dog' in img_name:
                y_label_list.append(1)
        x_data = np.array(img_data_list)
        y_label = np.array(y_label_list)
        print(x_data.shape)
        print(y_label.shape)
        return x_data, y_label


In [15]:
x_train,y_train= load_data(source='dog')

Loading from  C:\Users\jack_tsai\PycharmProjects\TensorTest\train
(500, 224, 224, 3)
(500,)


## Step 2: preprocessing data

In [16]:
def preprocessing_data(x_train,y_train,image_data_generator=False,num_classes=0,shuffled=True):
    if image_data_generator is False :
        # nomalize 如果有用ImageDataGenerator就不用先做preprocess_input
        x_train = preprocess_input(x_train)
    if num_classes > 0 :
        #turn y_train to one-hot
        y_train = np_utils.to_categorical(y_train, num_classes)
    if shuffled is True :
        #shuffle
        x_train,y_train = shuffle(x_train,y_train, random_state=2)
    return x_train,y_train

In [17]:
print("Data before ... x_train[0][0][0] = ",x_train[0][0][0])
print("Data before ... y_train[0] = ",y_train[0])
x_train,y_train = preprocessing_data(x_train,y_train,num_classes=NUM_CLASSES,image_data_generator=USE_IMAGE_GENERATOR)
print("Data After ... x_train[0][0][0] = ",x_train[0][0][0])
print("Data After ... y_train[0] = ",y_train[0])

Data before ... x_train[0][0][0] =  [1. 1. 1.]
Data before ... y_train[0] =  0
Data After ... x_train[0][0][0] =  [-20.939003 -34.779    -45.68    ]
Data After ... y_train[0] =  [1. 0.]


# Build Model 


## Step 1 : Load base model  and customize
watchout : include_top = true 會將最後fc層include進來，再設置的時候記得多用model.summary()來確保自已設定是否正確

In [18]:
def build_model() :
    image_input = Input(shape=(PIC_SIZE, PIC_SIZE, 3))
    base_model = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
    last_layer = base_model.get_layer('avg_pool').output
    X= Flatten(name='flatten')(last_layer)
    out = Dense(NUM_CLASSES, activation='softmax', name='output_layer')(X)
    model = Model(inputs=image_input,outputs= out)
    #Transfer_learning --設置model para trainable
    for layer in model.layers[:-1]: #[:-1]最後一個之前layer
        layer.trainable = False
        #只留最後的兩層layer來train
        #flatten沒有para!!
    model.summary()
    return model    

In [19]:
model=build_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [20]:
#注意loss fucntion，因為這邊是用classifier，所以選用crossentropy。不同應用可能用不同的function，甚至自定義
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.save("ResNet_pretrain.h5")

# Train Model

In [21]:
def get_data_generator():
    train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,# ((x/255)-0.5)*2  歸依化到±1之间
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    )
    val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    )
    train_generator = train_datagen.flow_from_directory(directory="C:\\Users\jack_tsai\PycharmProjects\TensorTest\\train1",
                                  target_size=(PIC_SIZE,PIC_SIZE),
                                  batch_size=BATCH_SIZE)
    val_generator = val_datagen.flow_from_directory(directory="C:\\Users\jack_tsai\PycharmProjects\TensorTest\\train2",
                                target_size=(PIC_SIZE,PIC_SIZE),
                                batch_size=BATCH_SIZE)
    return train_generator,val_generator

In [ ]:
def train_model(model,x_train_,y_train,image_data_generator=False):
    t=time.time()
    if image_data_generator is False :
        hist = model.fit(x_train,y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,validation_split=0.2)
    else :
        train_generator,val_generator = get_data_generator()
        hist = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=20,
                                 epochs=EPOCHS,
                                 validation_data=val_generator,
                                 validation_steps=12,
                                 class_weight='auto',
                                 shuffle = True)
            
    print('Training time: %s' % (t - time.time()))

In [ ]:
print("check data before training ")
print("x = ",x_train.shape)
print("y = ",y_train.shape)
print("x = ",x_train[0][0][0])
print("y = ",y_train[0])
train_model(model,x_train,y_train,image_data_generator=USE_IMAGE_GENERATOR)
model.save("cat_dog_ResNet.h5")

check data before training 
x =  (500, 224, 224, 3)
y =  (500, 2)
x =  [-20.939003 -34.779    -45.68    ]
y =  [1. 0.]
Train on 400 samples, validate on 100 samples
Epoch 1/12
384/400 [===========================>..] - ETA: 6s - loss: 0.4679 - acc: 0.7917 

In [ ]:
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(EPOCHS)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use('seaborn-whitegrid')
plt.show()|

In [ ]:
plt.figure(2,figsize=(5,3))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use('seaborn-whitegrid')
plt.show()